In [22]:
a = Dict("a" => 123, "b" => "123", "c" => 1e-10, "d" => "1e-10")

Dict{String, Any} with 4 entries:
  "c" => 1.0e-10
  "b" => "123"
  "a" => 123
  "d" => "1e-10"

In [23]:
smth = Meta.eval(Meta.parse("pi /2 "))

1.5707963267948966

---

## Загрузка необходимого файла и вызов функции по соответствующему запросу конфигурационного файла

In [24]:
function_file_path = "module_with_funcs.jl"
function_name = "func2"

"func2"

In [25]:
module_name = include(function_file_path)

LoadError: SystemError: opening file "c:\\Users\\artyo\\YandexDisk\\Other\\Julia\\BS_module\\Tests\\module_with_funcs.jl": No such file or directory

In [ ]:
func = getfield(module_name, Meta.parse(function_name))

func2 (generic function with 1 method)

In [ ]:
func(3)

9

---

## Считывание и обработка строковых значений из конфигурационного файла типа 1е-10, pi/2

In [ ]:
Config = Dict("a" => 123, "b" => "1e-10", "c" => 321, "d" => "pi/2")

Dict{String, Any} with 4 entries:
  "c" => 321
  "b" => "1e-10"
  "a" => 123
  "d" => "pi/2"

In [ ]:
for key = keys(Config)
    if(typeof(Config[key]) == String)
        Config[key] = eval(Meta.parse(Config[key]))
    end
end

In [ ]:
Config

Dict{String, Any} with 4 entries:
  "c" => 321
  "b" => 1.0e-10
  "a" => 123
  "d" => 1.5708

---

## Работа с YAML

In [ ]:
using YAML, Configurations
#include("Utils.jl")

In [ ]:
config_file_name = "config.yaml"

"config.yaml"

In [ ]:
data = ispath(config_file_name) ? YAML.load_file(config_file_name) : error("Configuration file " * string(config_file_name) * " does not exist!")

Dict{Any, Any} with 5 entries:
  "Task"          => "PlotODESolution"
  "ODE_File_Name" => "Utils\\BS_Cos_System.jl"
  "Int_setings"   => Dict{Any, Any}("reltol"=>1.0e-13, "maxiters"=>1.0e10, "tra…
  "Parameters"    => Dict{Any, Any}[Dict("name"=>"K", "Lname"=>"r\"K\"", "value…
  "SubTask"       => Dict{Any, Any}("TimeSeries"=>Dict{Any, Any}("proj"=>["proj…

In [ ]:
YAMLUtils.ParseConfig!(data)

Dict{Any, Any} with 4 entries:
  "Task"          => "Do_smth"
  "ODE_File_Name" => "Utils\\BS_Sin_System.jl"
  "Int_setings"   => Dict{Any, Any}("reltol"=>1.0e-13, "maxiters"=>1.0e10, "tra…
  "Parameters"    => Dict{Any, Any}[Dict("name"=>"a", "Lname"=>"r\"\\alpha\"", …

In [ ]:
YAML.write_file("testYAML.yaml", Dict("a" => 1, "b" => [1, 2, 3]))

---

## Время и даты

In [ ]:
using Dates

In [ ]:
time1 = now()

2022-11-01T12:34:14.199

In [ ]:
time2 = now()

2022-11-01T12:36:34.534

In [ ]:
Dates.format(time1, "yyyy/mm/dd HH:MM:SS")

"2022/11/01 12:34:14"

In [ ]:
print("Elapsed time : " * string(canonicalize(time2 - time1)))

Elapsed time : 2 minutes, 20 seconds, 335 milliseconds

---

## Создание функции системы ДУ с параемтрами заданными из конф. файла. Инетгрирование и вычисление Ляпуновских показателей

In [ ]:
using DifferentialEquations
using DynamicalSystems
using ForwardDiff

In [ ]:
function getSystem(dict::Dict)
    a = dict["a"]
    b = dict["b"]
    function System(dX, X, p, t)
        x1, x2 = X
        dX[1] = x1 + a
        dX[2] = x2 * b
        return SVector{2}(dX)
    end
end

getSystem (generic function with 1 method)

In [ ]:
config = Dict("a" => 2, "b" => 4)

Dict{String, Int64} with 2 entries:
  "b" => 4
  "a" => 2

In [ ]:
Sys1 = getSystem(config)

(::var"#System#74"{Int64, Int64}) (generic function with 1 method)

In [ ]:
include("Computation_Utils.jl")

Main.Computation_Utils

In [ ]:
function getCond(callback_func)

    function myCond(x, t, integrator)
        return callback_func(x)
    end
    return myCond
end

getCond (generic function with 1 method)

In [ ]:
myCond1 = getCond(x -> sin(x[1] - 0.25))

(::var"#myCond#88"{var"#89#90"}) (generic function with 1 method)

In [ ]:
myCond1(1, 2, 3)

0.6816387600233341

In [ ]:
sol = Computation_Utils.SolveODE(Sys1, [1, 1], 100; kwargs = Dict("reltol" => 1e-10), callback_func = myCond1)

LoadError: ArgumentError: reducing over an empty collection is not allowed

In [ ]:
lvDs = ContinuousDynamicalSystem(Sys1, [1.0, 2.0], [1.0, 2.0])

2-dimensional continuous dynamical system
 state:       [1.0, 2.0]
 rule f:      System
 in-place?    true
 jacobian:    ForwardDiff
 parameters:  [1.0, 2.0]

In [ ]:
lyapunovspectrum(lvDs, 2000)

2-element Vector{Float64}:
 3.910217249845047e-14
 2.9757621411337013e-14

In [ ]:
include("Utils\\BS_Uni_System.jl")

BS_Uni_Sys (generic function with 2 methods)

In [ ]:
function OuterFunc(SideFunc)
    function InnerFunc(dX, X, p, t)
        dX[1] = X[1] + SubFunc(p[1])
        dX[2] = X[2] + SideFunc(p[2])
        dX[1:2] .= dX[1:2] + [12, 34]
        return dX
    end
    return InnerFunc
end

function SubFunc(x)
    return x^2
end

SubFunc (generic function with 1 method)

In [ ]:
func = BS_Uni_Sys(x -> x^2)

(::var"#Uni_Sys!#71"{var"#72#73"}) (generic function with 1 method)

In [ ]:
func(zeros(6), 1.0 : 6.0, 1 : 5, 0)

6-element SVector{6, Float64} with indices SOneTo(6):
 41.42354138141536
 48.33666772782265
 46.71218191041544
 38.689757070869234
 48.57869928890133
 47.31398714249025

In [ ]:
lvDs = ContinuousDynamicalSystem(func, 1.0 : 6.0, 1 : 5)

6-dimensional continuous dynamical system
 state:       [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
 rule f:      Uni_Sys!
 in-place?    true
 jacobian:    ForwardDiff
 parameters:  [1, 2, 3, 4, 5]

In [ ]:
func = OuterFunc(x -> x^2)

(::var"#InnerFunc#32"{var"#35#36"}) (generic function with 1 method)

In [ ]:
func([1, 2], [3, 4], [5, 6], 0)

2-element Vector{Int64}:
 40
 74

In [ ]:
lvDs = ContinuousDynamicalSystem(OuterFunc(x -> x^2), zeros(2), zeros(2))

2-dimensional continuous dynamical system
 state:       [0.0, 0.0]
 rule f:      InnerFunc
 in-place?    true
 jacobian:    ForwardDiff
 parameters:  [0.0, 0.0]

In [ ]:
lyapunovspectrum(lvDs, 20)

6-element Vector{Float64}:
  1.2147253237836189
 -0.21588494762830343
 -5.735149014560517
 -7.117406117834933
 -7.7474014800812
 -9.474269769509423

---

## Создание переменной по имени и задание ей значения по данным из конфиг. файла

In [ ]:
config = Dict("name" => "a", "value" => 323)

Dict{String, Any} with 2 entries:
  "name"  => "a"
  "value" => 323

In [ ]:
someExpr = config["name"] * " = " * string(config["value"])

"a = 323"

In [ ]:
eval(Meta.parse(someExpr))

323

In [ ]:
print(a)

323

---

In [ ]:
using JLD

In [ ]:
SavePath = "Results\\test"

"Results\\test"

In [ ]:
mkpath(SavePath)

"Results\\test"

In [ ]:
save(SavePath * "\\" * "TaskName" * "Data.jld", "sol", Tosave)

In [ ]:
module_name = import(raw"ODEs\BS_Uni_System.jl")

ErrorException: syntax: invalid "import" statement: expected identifier

In [ ]:
config = Dict("a" => Dict("b" => 3))

Dict{String, Dict{String, Int64}} with 1 entry:
  "a" => Dict("b"=>3)

In [ ]:
config["a"]

Dict{String, Int64} with 1 entry:
  "b" => 3

In [ ]:
haskey(config["a"], "b")

true

In [ ]:
using PyCall

In [ ]:
pushfirst!(PyVector(pyimport("sys")."path"), "")
GU =  pyimport("Utils.Matlotlib_Utils")

PyObject <module 'Utils.Matlotlib_Utils' from 'c:\\Users\\artyo\\YandexDisk\\Other\\Julia\\BS_module\\Utils\\Matlotlib_Utils.py'>

In [ ]:
haskey(GU, "plt")

true

---

## Синтаксис kwargs...

In [ ]:
function f(s; kwargs...)
    println(s)
    for arg in kwargs
        println(arg)
    end
end

f (generic function with 1 method)

In [ ]:
f(12; Dict(:dt => 1e-3, :abstol => 1e-13, :reltol => 1e-13, :maxiters => 1e10)...)

12
:maxiters => 1.0e10
:reltol => 1.0e-13
:abstol => 1.0e-13
:dt => 0.001


In [ ]:
using DifferentialEquations
using DynamicalSystems

In [ ]:
prob_julia = ODEProblem(ODE_func, InitCond, (0.0, time_span))

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 500.0)
u0: 6-element Vector{Float64}:
 0.1
 0.2
 0.3
 0.4
 0.5
 0.6

In [ ]:
dict = Dict(:dt => 1e-3, :abstol => 1e-13, :reltol => 1e-13, :maxiters => 1e10)

Dict{Symbol, Float64} with 4 entries:
  :maxiters => 1.0e10
  :reltol   => 1.0e-13
  :abstol   => 1.0e-13
  :dt       => 0.001

In [ ]:
sol_julia = solve(prob_julia, DP8(); save_everystep = false,  dict...)

retcode: Success
Interpolation: 1st order linear
t: 2-element Vector{Float64}:
   0.0
 500.0
u: 2-element Vector{Vector{Float64}}:
 [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
 [-3.986853058938782e-14, -6.277999214383971e-14, -1.1576742808529368e-14, -6.405918435627489e-14, 4.464418579701838e-14, -4.34401746978249e-14]

---

In [ ]:
using YAML
using Configurations
using Latexify, LaTeXStrings, SymEngine

In [ ]:
## Importing utils ##
#include("Utils\\Computation_Utils.jl")
include("..\\Utils\\YAML_Utils.jl")

Main.YAML_Utils

In [ ]:
config_dict = YAML.load_file("..\\config.yaml")

Dict{Any, Any} with 4 entries:
  "Session_Input"  => Dict{Any, Any}("Tasks_List"=>Dict{Any, Any}[Dict("Task_In…
  "Session_folder" => "Results"
  "Session_name"   => "Some_Tasks"
  "Session_Output" => "Output.txt"

In [ ]:
config_dict["Session_Input"]["Tasks_List"][1]

Dict{Any, Any} with 4 entries:
  "Task_Input"  => Dict{Any, Any}("ODESolver_settings"=>Dict{Any, Any}("init_co…
  "Task_Func"   => "SolveODE"
  "Task_Output" => Dict{Any, Any}("Data_file_name"=>"SolveODE_Data.jld")
  "Task_File"   => "Tasks\\BS_ODE_Solving.jl"

In [ ]:
# Required fileds
module_name = include(config_dict["ODE_File_Name"])
ODE_func = getfield(module_name, Meta.parse(config_dict["ODE_Func_Name"]))(YAML_Utils.MakeParametersDict(config_dict), eval(Meta.parse(config_dict["Couple_func"])))
InitCond = config_dict["ODESolver_setings"]["InitCond"]
time_span = config_dict["ODESolver_setings"]["time_span"]
println("Required fileds procceed")

# Optional fields
ODE_alg = haskey(config_dict["ODESolver_setings"], "alg") ? config_dict["ODESolver_setings"]["alg"] : nothing
ParsedKwargs = haskey(config_dict["ODESolver_setings"], "kwargs") ? YAML_Utils.ParseODEKwargs(config_dict["ODESolver_setings"]["kwargs"]) : nothing
trans_time = haskey(config_dict["ODESolver_setings"], "trans_time") ? config_dict["ODESolver_setings"]["trans_time"] : nothing
callback = haskey(config_dict["ODESolver_setings"], "callback") ? Callback_Utils.ParseAnonCondition(eval(Meta.parse(config_dict["ODESolver_setings"]["callback"]))) : nothing
println("Optional fileds procceed")

Required fileds procceed
Optional fileds procceed


In [ ]:
sol = Computation_Utils.SolveODE(ODE_func, InitCond, time_span;
                                 alg = ODE_alg,
                                 trans_time = trans_time,
                                 kwargs = ParsedKwargs,
                                 callback_func = callback);
println("SolveODE succeed")

SolveODE succeed


In [ ]:
## Plot results ##
Graphics_Utils.PlotRequestedGraphics(sol, config_dict, "Results")

---

In [ ]:
using JLD2

In [ ]:
cd(raw"C:\Users\artyo\YandexDisk\Other\Julia\BS_module")

In [ ]:
struct py_sol
    t::Vector{Float64}
    y::Vector{Vector{Float64}}
    t_events::Vector{Vector{Float64}}
    y_events::Vector{Matrix{Float64}}
end  

In [ ]:
file_name = raw"Results\Some_Tasks 2023_01_11 16_23_05\SolveODE_Data.jld2"

"Results\\Some_Tasks 2023_01_11 16_23_05\\SolveODE_Data.jld2"

In [ ]:
sol = load_object(file_name)

┌ Warning: type Main.BS_ODE_Tasks.Computation_Utils.py_sol does not exist in workspace; reconstructing
└ @ JLD2 C:\Users\artyo\.julia\packages\JLD2\1YVED\src\data\reconstructing_datatypes.jl:495


JLD2.ReconstructedTypes.var"##Main.BS_ODE_Tasks.Computation_Utils.py_sol#316"([0.0, 0.1, 0.43865182679061143, 0.7797840935322946, 1.1963633280703436, 1.7069479753783061, 2.523782834235203, 3.488455367373043, 4.276776310585493, 4.940957847617945  …  99.41612526133575, 99.48277449841709, 99.54840841692824, 99.61487634838602, 99.67857812690899, 99.74261007709588, 99.80800655710146, 99.87531713039517, 99.94491857790753, 100.0], [[6.283185309326366, 6.283185309689658, 6.283185311440916, 6.283185314440682, 6.283185321097774, 6.2831853380739195, 6.283185417799717, 6.283185806090078, 6.283187015675529, 6.2831901268851835  …  15.707963267951008, 15.70796326795109, 15.707963267951163, 15.707963267951232, 15.70796326795129, 15.707963267951342, 15.707963267951392, 15.707963267951436, 15.70796326795148, 15.70796326795151], [12.5663705908434, 12.566370607661389, 12.566370629907343, 12.566370634420014, 12.566370635389632, 12.56637063553333, 12.566370635547313, 12.566370635547596, 12.56637063554747, 1

In [ ]:
sol.t

673-element Vector{Float64}:
   0.0
   0.1
   0.43865182679061143
   0.7797840935322946
   1.1963633280703436
   1.7069479753783061
   2.523782834235203
   3.488455367373043
   4.276776310585493
   4.940957847617945
   5.51744413643272
   6.0282027866064425
   6.535842053766405
   ⋮
  99.32453270347193
  99.36610150660192
  99.41612526133575
  99.48277449841709
  99.54840841692824
  99.61487634838602
  99.67857812690899
  99.74261007709588
  99.80800655710146
  99.87531713039517
  99.94491857790753
 100.0

---

In [ ]:
module abc
export a, f

a = 123
b = "123"

function f()
    return "That's a function"
end
end

Main.abc

In [ ]:
str = "abc"
eval(Meta.parse("using ." * str))

---

In [ ]:
str = "..\\Tasks\\BS_ODE_Solving.jl"

"..\\Tasks\\BS_ODE_Solving.jl"

In [ ]:
include(str)

Main.BS_ODE_Solving

In [ ]:
findlast(".", str)[1]

24

In [ ]:
findlast("\\", str)

9:9

In [ ]:
str[findlast("\\", str)[1] + 1 : findlast(".", str)[1] - 1]

"BS_ODE_Solving"

In [ ]:
using .BS_ODE_Solving

In [ ]:
a = 321
"12321  $(a)  342342"

"12321  321  342342"

---

In [19]:
mutable struct Poly_solve_Task{T <: Real}
    coefs::Vector{T}
end

function (p::Poly_solve_Task)()
    a, b, c = p.coefs
    D = b^2 - 4 * a * c
    if D > 0
        return [(-b + sqrt(D))/2, (-b - sqrt(D))/2]
    elseif D == 0
        return (-b + sqrt(D))/2
    else
        return "No real roots"
    end
end

function GetConfig(p::Poly_solve_Task)
    return Dict(:coefs => p.coefs)
end

GetConfig (generic function with 1 method)

In [16]:
pol = Poly_solve_Task([1, 2, -3])

Poly_solve_Task{Int64}([1, 2, -3])

In [17]:
pol()

2-element Vector{Float64}:
  1.0
 -3.0

In [20]:
GetConfig(pol)

Dict{Symbol, Vector{Int64}} with 1 entry:
  :coefs => [1, 2, -3]

In [ ]:
abstract type ODE end

In [1]:
mutable struct A
    a::Float64
    b::Float64
    c::Union{Float64, Nothing}
    function A(v::Vector{Float64})
        new(v[1], v[2], v[3])
    end
    function A(a, b, c)
        new(a, b, c)
    end
end

In [2]:
function Base.string(a::A)
    return "A 123"
end

In [14]:
function Base.show(io::IO, a::A)
    print(io, "123 A")
end

In [1]:
abstract type A_1 end
mutable struct A <: A_1
    x::Float64
end

function smth(a::A_1)
    println("value of field x in A = $(a.x)")
end

smth (generic function with 1 method)

In [2]:
a = A(1.0)
smth(a)

value of field x in A = 1.0


In [10]:
mutable struct nextA <: A_1
    y::A
end

function smth(a::nextA)
    smth(a.y)
end

smth (generic function with 2 methods)

In [11]:
nexta = nextA(a)
smth(nexta)

value of field x in A = 1.0


In [8]:
a = A(1.0)

A(1.0)

In [11]:
b = B(1.0)

LoadError: MethodError: no method matching B(::Float64)
[0mClosest candidates are:
[0m  B([91m::T[39m) where T<:A_1 at In[7]:3

In [16]:
struct MySavedValues{T, S}
    t::Vector{T}
    s::Vector{S}
end

function MySavedValues(::T, ::S) where {T, S}
    println(T)
    println(S)
    MySavedValues{T, S}(Vector{T}(), Vector{S}())
end

MySavedValues

In [17]:
MySavedValues(Float64, NTuple{6, Float64})

Float64
NTuple{6, Float64}


MySavedValues{Float64, NTuple{6, Float64}}(Float64[], NTuple{6, Float64}[])

In [ ]:
abstract type ODEType end

mutable struct Abstract_ODE <: ODEType
    ODE_Sys::Function
    ODE_Parameters::Dict{String, Any}
    Couple_function::String
end

In [ ]:


mutable struct BS_ODE <: ODEType
    ODE_Sys::Function
    ODE_Parameters::Dict{String, Any}
    Couple_function::String
end

function Get_BS_ODE(bs::BS_ODE)
    ODE_rhs = bs.ODE_Sys(bs.ODE_Parameters, eval(Meta.parse(bs.Couple_function)))
    return ODE(dX, u, p, t) = Base.invokelatest(ODE_rhs, dX, u, p, t)
end

function Base.show(io::IO, bs::BS_ODE)
    for property in propertynames(bs)
        println(io, "[$property]: $(getproperty(bs, property))")
    end
end

In [1]:
function f()
    println("123")
end

f (generic function with 1 method)

In [8]:
mutable struct S
    a::Int32

    function S(a::Float64)
        println(a)
        S("1")
    end

    function S(b::String)
        println(b)
        new(1)
    end
end

In [13]:
mutable struct S{T<:Any}
    a::Int32

    function S{T}(a::Int32) where T

        new{T}(a)
    end
end

In [1]:
using matplotlib

LoadError: ArgumentError: Package matplotlib not found in current path.
- Run `import Pkg; Pkg.add("matplotlib")` to install the matplotlib package.

In [2]:
using PyPlot

In [8]:
matplotlib.use("")

LoadError: PyError ($(Expr(:escape, :(ccall(#= C:\Users\artyo\.julia\packages\PyCall\ygXW2\src\pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ValueError'>
ValueError("'' is not a valid value for backend; supported values are ['GTK3Agg', 'GTK3Cairo', 'GTK4Agg', 'GTK4Cairo', 'MacOSX', 'nbAgg', 'QtAgg', 'QtCairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']")
  File "C:\Users\artyo\.julia\conda\3\lib\site-packages\matplotlib\__init__.py", line 1140, in use
    name = validate_backend(backend)
  File "C:\Users\artyo\.julia\conda\3\lib\site-packages\matplotlib\rcsetup.py", line 255, in validate_backend
    else _validate_standard_backends(s))
  File "C:\Users\artyo\.julia\conda\3\lib\site-packages\matplotlib\rcsetup.py", line 82, in __call__
    raise ValueError(msg)
